# Desease Diagnose

## Etapa 03

Nosso trabalho consiste em auxiliar o médico na hora de diagnosticar seus
pacientes.  
Através de um banco de dados, que será alimentado, nosso sistema irá indicar
probabilidades de diagnósticos, ou seja, qual doença é mais possível que o paciente tenha
contraído.  
Após a análise o médico poderá avaliar, ou pedir um exame, para confirmar o
diagnóstico. Caso essa avaliação não confirme a doença com mais probabilidade o médico
poderá alimentar o sistema com essa informação e permitir com que as probabilidades
sejam recalculadas a partir daquela informação.  


## Modelo Conceitual 

<img src="etapa03-modelo.png">

## Modelagem Lógica 


Doenca (codigo,nome)

Sintomas (codigo,nome)

Paciente(ID, nome,nascimento,celular,localidade)

Fator(Codigo,Fator,Tipo)  

Diagnostico (ID, doencaCodigo, data)  
    &emsp; CHE: doencaCodigo -> Doenca (codigo)

PacienteSintomas(pacienteID, sintomaCodigo)   
    &emsp; CHE: IDpaciente -> Paciente (ID)  
    &emsp; CHE: CodSintoma -> Sintomas (codigo)  
  
DoencaSintomas (doencaCodigo, sintomaCodigo)  
    &emsp; CHE: doencaCodigo -> Doenca (codigo)  
    &emsp; CHE: sintomaCodigo -> Sintomas (codigo)  
  
PacienteDiagnostico (pacienteID, diagnosticoID)  
    &emsp; CHE: pacienteID -> Paciente (ID)  
    &emsp; CHE: diagnosticoID -> Diagnostico(ID)  
  
FatorDoenca(CodFator,Dcode,Prob)  
    &emsp; CHE: CodFator -> Fator(Codigo)  
    &emsp; CHE: Dcode -> Doenca(codigo)  
  
FatorPaciente(CodFator,IDpaciente)  
    &emsp; CHE: CodFator -> Fator(Codigo)  
    &emsp; CHE: IDpaciente -> Paciente(ID)  


## Códigos para criação de dados em Cypher

In [ ]:
// Criar Diagnostico
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/leferlin/disease-diagnose/master/data/Diagnostico.csv" AS row
CREATE (:Diagnostico {ID: row.ID, doencaCodigo: row.doencaCodigo, data: row.data});

// Criar Doenca
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/leferlin/disease-diagnose/master/data/Doenca.csv" AS row
CREATE (:Doenca {codigo: row.codigo, nome: row.nome});

// Criar Fator
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/leferlin/disease-diagnose/master/data/Fator.csv" AS row
CREATE (:Fator {Codigo: row.Codigo, Fator: row.Fator, Tipo: row.Tipo});

// Criar Paciente
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/leferlin/disease-diagnose/master/data/Paciente.csv" AS row
CREATE (:Paciente {ID: row.ID, nome: row.nome, nascimento: row.nascimento, celular: row.celular, localidade: row.localidade});

// Criar Sintomas
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/leferlin/disease-diagnose/master/data/Sintomas.csv" AS row
CREATE (:Sintomas {codigo: row.codigo, nome: row.nome});

// Criar FatorDoenca
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/leferlin/disease-diagnose/master/data/FatorDoenca.csv' AS line
MATCH (d:Fator {Codigo: line.CodFator})
MATCH (p:Doenca {codigo: line.Dcode})
CREATE (d)-[:FatorDoenca {Probabilidade: line.Prob}]->(p)

// Criar FatorPaciente
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/leferlin/disease-diagnose/master/data/FatorPaciente.csv' AS line
MATCH (d:Fator {Codigo: line.CodFator})
MATCH (p:Paciente {ID: line.Idpaciente})
CREATE (p)-[:FatorPaciente]->(d)

// Criar DoencaSintoma
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/leferlin/disease-diagnose/master/data/DoencaSintomas.csv' AS line
MATCH (d:Doenca {codigo: line.doencaCodigo})
MATCH (p:Sintomas {codigo: line.sintomaCodigo})
CREATE (p)-[:DoencaSintoma {Ocorrencias: line.Ocorrencias}]->(d)

// Criar PacienteSintoma
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/leferlin/disease-diagnose/master/data/PacienteSintoma.csv' AS line
MATCH (d:Paciente {ID: line.Idpaciente})
MATCH (p:Sintomas {codigo: line.CodSintoma})
CREATE (d)-[:PacienteSintoma]->(p)

// Criar PacienteDiagnostico
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/leferlin/disease-diagnose/master/data/PacienteDiagnostico.csv' AS line
MATCH (d:Paciente {ID: line.pacienteID})
MATCH (p:Diagnostico {ID: line.diagnosticoID})
CREATE (d)-[:PacienteDiagnostico]->(p)






#Consultas

In [ ]:
// Probabilidade segundo os fatores de risco (exclusivamente) do Paciente 'n' ter contraido a Doenca 'd'
MATCH p = (n:Paciente{ID: 'patOJATVU'})-[r:FatorPaciente]->(m:Fator)-[t:FatorDoenca]->(d:Doenca{nome: 'hypertensive disease'})
RETURN AVG(toFloat(replace(t.Probabilidade, ",", ".")))

// Caminho Paciente Y -> Fatores de risco respectivos -> Doencas
MATCH p = (n:Paciente{ID: 'patOJATVU'})-[r:FatorPaciente]->(m:Fator)-[t:FatorDoenca]->(d:Doenca{nome: 'hypertensive disease'})
RETURN p